# Circulation méridienne de Hadley (données ERA5)

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2022**

Les fichiers de données au format netcdf (moyennes mensuelles ERA5 du vent méridien et de la vitesse verticale) doivent être récupérés et placés dans le répertoire data :

- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means?tab=form

Fichiers à retraiter avec CDO (https://code.mpimet.mpg.de/projects/cdo) pour baisser la résolution spatiale à 2° et réduire la taille des fichiers :

- cdo remapbil,r180x90 era5_v.1979-2018.mon.mean.nc era5_v.1979-2018.mon.mean.2deg.nc
- cdo remapbil,r180x90 era5_w.1979-2018.mon.mean.nc era5_w.1979-2018.mon.mean.1deg.nc

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz

# Traitement des données de vent méridien et de vitesse verticale

In [ ]:
diri="./monthly_era5/"
fv    = xr.open_dataset(diri+"era5_v.1979-2018.mon.mean.2deg.nc")
fw    = xr.open_dataset(diri+"era5_w.1979-2018.mon.mean.2deg.nc")
print(fv)
print(fw)

In [ ]:
year1='1978'
year2='2018'

In [ ]:
fv    = xr.open_dataset(diri+"era5_v.1979-2018.mon.mean.2deg.nc").sel(time=slice(year1,year2)).sel(level=slice(100,1000))
fw    = xr.open_dataset(diri+"era5_w.1979-2018.mon.mean.2deg.nc").sel(time=slice(year1,year2)).sel(level=slice(100,1000))

lat  = fv.lat.values
lev = fv.level.values

print(lat)
print(lev)

In [ ]:
seasons=['DJF','JJA','MAM','SON']
months=['January','February','March','April', 'May', 'June', 'July',
        'August', 'September', 'October', 'November', 'December']

# moyenne saisonnière
fv_mean = fv.groupby('time.season').mean('time')
fw_mean = fw.groupby('time.season').mean('time')
v_season = fv_mean['v']
w_season = fw_mean['w']

# moyenne mensuelle
fv_mean_month = fv.groupby('time.month').mean('time')
v_month = fv_mean_month['v']

# moyenne zonale
vz_season = v_season.mean('lon')
wz_season = w_season.mean('lon')
vz_month = v_month.mean('lon')

# moyenne annuelle
vz_annual=vz_season.mean('season')
wz_annual=wz_season.mean('season')

print(vz_annual)
print(vz_month)
print(vz_annual)

# Tracés

In [ ]:
levels_wz = np.arange(-0.05,0.052,0.002)
levels_vz =[-7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0,
 -2.5, -2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0, 2.5,
  3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7]

In [ ]:
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 100)
    ax.set_xlim(-90, 90)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10)) 
    return ax

In [ ]:
fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(1, 1, 1)
fig.suptitle('Meridional wind (m/s) and vertical velocity : ERA5 '+year1+'-'+year2, fontsize=16)

ax.set_title('Annual mean', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, wz_annual[:,:], levels_wz, cmap='seismic', extend='both')
c = ax.contour(lat, lev, vz_annual[:,:], levels_vz, colors='black', linewidths=1)
plt.clabel(c, levels_vz, fmt='%1.1f')

cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_annual_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(15, 7), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Vertical velocity (Pa/s) and meridional wind (m/s) - zonal mean : ERA5 '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
	
 ax.set_title(seasons[i], fontsize=14)
 plot_zonal_mean(ax)
 cf = ax.contourf(lat, lev, wz_season[i,:,:], levels_wz, cmap='seismic', extend='both')
 c = ax.contour(lat, lev, vz_season[i,:,:], levels_vz, colors='black', linewidths=1)
 plt.clabel(c, levels_vz, fmt='%1.1f')

cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Diagnostic de la fonction de courant méridienne

Le flux élémentaire de masse vers le nord (unité $kg.s^{-1}$) à travers une ligne de latitude $\phi$ s'écrit :

$$ d\psi = a \cos\phi d\lambda ~ v ~ \frac{dp}{g} $$


$a$ : rayon de la Terre ; $g$ : accélération de la gravité


Ainsi, le flux de masse vers le nord total à travers $\phi$ au-dessus d'un niveau de pression $p$ est donné par :

\begin{align*} 
\psi(p) &= \int_0^{2\pi} \int_0^p a \cos\phi d\lambda ~ v ~ \frac{dp}{g} \\
&= 2\pi a \frac{\cos\phi}{g} \int_0^p [v] dp 
\end{align*}

L'équation de continuité (conservation de la masse) en coordonnée pression et en moyenne zonale s'écrit :

$$ \frac{1}{a \cos\phi} \frac{\partial}{\partial \phi} \left( [\overline{v}] \cos\phi \right) + \frac{\partial [\overline{\omega}]}{\partial p} = 0 $$

Ainsi, le vecteur de composantes $[\overline{v}], [\overline{\omega}]$ est non-divergent. On peut donc l'interpréter par une fonction de courant scalaire $\psi$, la **fonction de courant de transport de masse**:

\begin{align*}
2\pi a \cos\phi [\overline{v}] &= \frac{\partial \psi}{\partial p} \\
2\pi a^2 \cos\phi \left( [\overline{\omega}]\right) = -\frac{\partial \psi}{\partial \phi}
\end{align*}

cohérente avec la définition de $\psi$ ci dessus

On peut donc calculer et tracer la fonction de courant uniquement à partir de données de vent méridien à partir de la formule :

$$ \psi(lev,lat) = \frac{2\pi a \cos\phi}{g} \int_0^p [\overline{v}](lev,lat) dp $$

Les contours de $\psi$ peuvent être interprétés comme des isolignes de la **circulation méridienne moyenne**, c'est-à-dire, la circulation de masse en moyenne zonale et temporelle dans un plan latitude-pression.

<div class="alert alert-danger">
<b>Utiliser la fonction integrate.cumtrapz de scipy ou la méthode cumulative_integrate de xarray pour implémenter le calcul de la fonction de courant méridienne $\psi$.</b>
    
- https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html
- https://docs.scipy.org/doc/scipy-1.4.1/reference/generated/scipy.integrate.cumtrapz.html
    
<br>
    
$\psi(lev,lat)=\frac{2\pi a cos(lat)}{g}\int_0^P [v](lev,lat)dP$

</div>

In [ ]:
coslat=np.cos(lat[np.newaxis,np.newaxis,:]*np.pi/180.)

# Tracés

In [ ]:
levels_psi = np.arange(-18,19,1)
levels_psi_an = np.arange(-10,11,1)
levels_psi2 =[-18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7,
 -6, -5, -4, -3, -2, -1, 1, 2, 3, 4,
  5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [ ]:
fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(1, 1, 1)
fig.suptitle('Mass meridional streamfunction : ERA5 '+year1+'-'+year2, fontsize=16)

ax.set_title('Annual mean', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, psi_annual[:,:], levels_psi_an, cmap='PuOr_r', extend='both')
c = ax.contour(lat, lev, psi_annual[:,:], levels_psi2, colors='black', linewidths=1)
plt.clabel(c, levels_psi2, fmt='%2.1i')

cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
cb.set_label('10$^{10}$ kg/s', size='small')

figname='./figs/psi_zmean_annual_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(15, 7), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Mass meridional streamfunction : ERA5 '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean(ax)
    cf = ax.contourf(lat, lev, psi_season[i,:,:], levels_psi, cmap='PuOr_r', extend='both')
    c = ax.contour(lat, lev, psi_season[i,:,:], levels_psi2, colors='black', linewidths=1)
    plt.clabel(c, levels_psi2, fmt='%2.1i')

cb = fig.colorbar(cf, orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('10$^{10}$ kg/s', size='small')

figname='./figs/psi_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
for i in range(12): 
    #print(months[i])
    fig = plt.figure(figsize=(15., 8.))
    fig.suptitle('Mass meridional streamfunction : ERA5 '+year1+'-'+year2, fontsize=16)
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(months[i], fontsize=14)
    plot_zonal_mean(ax)
    
    cf = ax.contourf(lat, lev, psi_month[i,:,:], levels_psi, cmap='PuOr_r', extend='both')
    c = ax.contour(lat, lev, psi_month[i,:,:], levels_psi2, colors='black', linewidths=1)
    plt.clabel(c, levels_psi2, fmt='%2.1i')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
    cb.set_label('10$^{10}$ kg/s', size='small')
    
    figname='./anim/psi_zmean_monclim_'+str(i).zfill(2)
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
gif_filepath = './anim/psi_zmean_monclim.gif'
make_animation(gif_filepath)